In [27]:
import requests
from bs4 import BeautifulSoup

base_url = "https://pandas.pydata.org/docs/reference/index.html"
response = requests.get(base_url)

In [28]:
soup = BeautifulSoup(response.text, "lxml")

In [29]:
ref_url = "https://pandas.pydata.org/docs/reference/"

In [30]:
l1_elems = soup.find_all(class_="toctree-l1")
req_l1_elems = l1_elems[-len(l1_elems)+16:]

In [31]:
first_level = {}
curr_parent = ""
for parent_functions in req_l1_elems:
    for func in parent_functions.find_all('a'):
        href = func['href']
        if "#" not in href and "api/pandas" not in href:
            first_level.update({href:{"functions":[],"name":func.text,"url":ref_url+href}})
            curr_parent = href
        else:
            first_level[curr_parent]['functions'].append({
                "name":func.text,
                "url":ref_url+href
            })

In [32]:
first_level['plotting.html']['functions']
# https://pandas.pydata.org/docs/reference/io.html

[{'name': 'pandas.plotting.andrews_curves',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.andrews_curves.html'},
 {'name': 'pandas.plotting.autocorrelation_plot',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.autocorrelation_plot.html'},
 {'name': 'pandas.plotting.bootstrap_plot',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.bootstrap_plot.html'},
 {'name': 'pandas.plotting.boxplot',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.boxplot.html'},
 {'name': 'pandas.plotting.deregister_matplotlib_converters',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.deregister_matplotlib_converters.html'},
 {'name': 'pandas.plotting.lag_plot',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.lag_plot.html'},
 {'name': 'pandas.plotting.parallel_coordinates',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.parallel_coordinates.html'},


## EXTRACTING FUNCTIONS

In [33]:
# parent_url = first_level['io.html']['url']

# parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
# url_id = "pickling"
# s1 = parent_soup.find(attrs={"id":url_id})
# s1

In [34]:
# first_level['io.html']['functions']

In [35]:
first_level['plotting.html']

{'functions': [{'name': 'pandas.plotting.andrews_curves',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.andrews_curves.html'},
  {'name': 'pandas.plotting.autocorrelation_plot',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.autocorrelation_plot.html'},
  {'name': 'pandas.plotting.bootstrap_plot',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.bootstrap_plot.html'},
  {'name': 'pandas.plotting.boxplot',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.boxplot.html'},
  {'name': 'pandas.plotting.deregister_matplotlib_converters',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.deregister_matplotlib_converters.html'},
  {'name': 'pandas.plotting.lag_plot',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.lag_plot.html'},
  {'name': 'pandas.plotting.parallel_coordinates',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.pa

In [38]:
def get_links(id_elem,base_func_url,class_name,first_level_name,url_id):
    curr_urls = []
    try:
        func_urls = id_elem.find_all(attrs={"class":class_name})
        for odd_url in func_urls:
            try:
                func_url = odd_url.find('a')['href']
                curr_urls.append(base_func_url+func_url)
            except Exception as e:
                print(e)
                print(first_level_name)
                print(odd_url)
                print(url_id)
                print('-'*100)
        return curr_urls
    except Exception as e:
        print(e,url_id)
        func_urls = id_elem.find(attrs={"class":class_name}).find('a')['href']
        curr_urls.append(base_func_url+func_url)
        return curr_urls
    finally:
        return curr_urls
    
base_func_url = "https://pandas.pydata.org/docs/reference/"
for first_level_name in first_level:
    parent_url = first_level[first_level_name]['url']
    parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
    for idx,func in enumerate(first_level[first_level_name]['functions']):
        url_id = func['url'].split("#")[-1]
        id_elem = parent_soup.find(attrs={"id":url_id})
        odd_urls = get_links(id_elem,base_func_url,"row-odd",first_level_name,url_id)     
        even_urls = get_links(id_elem,base_func_url,"row-even",first_level_name,url_id)     
        # all_urls.extend(odd_urls)
        # all_urls.extend(even_urls)
        if odd_urls == [] and even_urls == []:
            function_urls = [first_level[first_level_name]['functions'][idx]['url']]
        else:
            function_urls = odd_urls + even_urls
        first_level[first_level_name]['functions'][idx]['function_urls'] = function_urls

'NoneType' object is not subscriptable
series.html
<tr class="row-odd"><th class="head"><p>Data Type</p></th>
<th class="head"><p>Accessor</p></th>
</tr>
accessors
----------------------------------------------------------------------------------------------------
'NoneType' object is not subscriptable
arrays.html
<tr class="row-odd"><th class="head"><p>Kind of Data</p></th>
<th class="head"><p>pandas Data Type</p></th>
<th class="head"><p>Scalar</p></th>
<th class="head"><p>Array</p></th>
</tr>
objects
----------------------------------------------------------------------------------------------------
'NoneType' object is not subscriptable
arrays.html
<tr class="row-odd"><th class="head"><p>PyArrow type</p></th>
<th class="head"><p>pandas extension type</p></th>
<th class="head"><p>NumPy type</p></th>
</tr>
objects
----------------------------------------------------------------------------------------------------
'NoneType' object has no attribute 'find_all' https://pandas.pydata.org

In [39]:
first_level['plotting.html']['functions']

[{'name': 'pandas.plotting.andrews_curves',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.andrews_curves.html',
  'function_urls': ['https://pandas.pydata.org/docs/reference/api/pandas.plotting.andrews_curves.html']},
 {'name': 'pandas.plotting.autocorrelation_plot',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.autocorrelation_plot.html',
  'function_urls': ['https://pandas.pydata.org/docs/reference/api/pandas.plotting.autocorrelation_plot.html']},
 {'name': 'pandas.plotting.bootstrap_plot',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.bootstrap_plot.html',
  'function_urls': ['https://pandas.pydata.org/docs/reference/api/pandas.plotting.bootstrap_plot.html']},
 {'name': 'pandas.plotting.boxplot',
  'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.boxplot.html',
  'function_urls': ['https://pandas.pydata.org/docs/reference/api/pandas.plotting.boxplot.html']},
 {'name': 'pandas.plotting.dere

In [42]:
from tqdm import tqdm
import logging
from IPython.display import clear_output
logname = "pandas.log"
# logging.basicConfig(filename=logname,
#                     filemode='a',
#                     format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
#                     datefmt='%H:%M:%S',
#                     level=logging.DEBUG)

def get_param_data(first_level):
    not_worked = []
    for parent in first_level:
        parent_dict = first_level[parent]['functions']
        for idx,sub_level in enumerate(parent_dict):
            parent_dict[idx].update({"function_definitions":[]})
            for func_url in tqdm(sub_level['function_urls']):
                func_response = requests.get(func_url)
                func_soup = BeautifulSoup(func_response.content, "lxml",from_encoding="utf-8")
                
                func_name = func_soup.find("h1").text.replace("#","") #remove #
                elem = func_soup.find(attrs={"class":"sig sig-object py"})
                try:
                    full_function = elem.text.replace("[source]#","").replace("\n","")
                    func_text = func_soup.find("dd").find('p').text
                    curr_dict = {"function_name":func_name,"full_function":full_function,"function_text":func_text,"parameter_names_desc":[],"function_url":func_url}
                    em = func_soup.find_all(attrs={"class":"field-odd"})
                    if em[0].text =='Parameters:':
                        param_names = em[-1].find_all("dt")
                        desc_list = em[-1].find_all("dd")

                        # if len(param_names)!=len(desc_list):
                        #     print(func_name)
                        #     print(f"param_names={len(param_names)} and desc_list={len(desc_list)}")
                        # first_level[parent]['function_definitions'].append()
                        for pn,dn in zip(param_names,desc_list):
                            try:
                                param_name = pn.strong.text
                                param_type = pn.find(attrs={"class":"classifier"}).text
                                if param_name == "**kwargs": 
                                    continue
                                param_desc = dn.text
                                curr_dict['parameter_names_desc'].append({"param_name":param_name,"param_type":param_type,"param_desc":param_desc})
                            except Exception as e:
                                print(e,pn.text)
                    # else:
                    #     parent_dict[idx]['function_definitions'].append(curr_dict)
                    #     continue
                except Exception as e:
                    # print(e)
                    not_worked.append((func_name,func_text,e,func_url))
                    # logging.debug("With error {e} for {func_name} and {func_text}")
                finally:
                    parent_dict[idx]['function_definitions'].append(curr_dict)
        clear_output(wait=True)
    return first_level,not_worked

function_def_dict,not_worked = get_param_data(first_level)

100%|██████████| 1/1 [00:00<00:00,  8.06it/s]


In [43]:
first_level['plotting.html']

{'functions': [{'name': 'pandas.plotting.andrews_curves',
   'url': 'https://pandas.pydata.org/docs/reference/api/pandas.plotting.andrews_curves.html',
   'function_urls': ['https://pandas.pydata.org/docs/reference/api/pandas.plotting.andrews_curves.html'],
   'function_definitions': [{'function_name': 'pandas.plotting.andrews_curves',
     'full_function': 'pandas.plotting.andrews_curves(frame, class_column, ax=None, samples=200, color=None, colormap=None, **kwargs)',
     'function_text': 'Generate a matplotlib plot for visualizing clusters of multivariate data.',
     'parameter_names_desc': [{'param_name': 'frame',
       'param_type': 'DataFrame',
       'param_desc': 'Data to be plotted, preferably normalized to (0.0, 1.0).\n'},
      {'param_name': 'class_column',
       'param_type': 'label',
       'param_desc': 'Name of the column containing class names.\n'},
      {'param_name': 'ax',
       'param_type': 'axes object, default None',
       'param_desc': 'Axes to use.\n'},
 

In [44]:
len(not_worked)

894

In [45]:
with open('not_worked.txt', 'w') as file:
    for item in not_worked:
        file.write(f"{item}\n")

In [46]:
import json
filename = "pandas_function_v3.json"

# Serialize the dictionary to a JSON string
json_data = json.dumps(function_def_dict, ensure_ascii=False)

# Write the JSON string to a file with UTF-8 encoding
with open(filename, "w", encoding="utf-8") as json_file:
    json_file.write(json_data)

print("Data has been successfully written to", filename)

Data has been successfully written to pandas_function_v3.json


## CLEANING DATA

In [47]:
import json
filename = "pandas_function_v3.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        data = json.load(json_file)

In [48]:
data['io.html']

{'functions': [{'name': 'Pickling',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#pickling',
   'function_urls': ['https://pandas.pydata.org/docs/reference/api/pandas.read_pickle.html#pandas.read_pickle',
    'https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_pickle.html#pandas.DataFrame.to_pickle'],
   'function_definitions': [{'function_name': 'pandas.read_pickle',
     'full_function': "pandas.read_pickle(filepath_or_buffer, compression='infer', storage_options=None)",
     'function_text': 'Load pickled pandas object (or any object) from file.',
     'parameter_names_desc': [{'param_name': 'filepath_or_buffer',
       'param_type': 'str, path object, or file-like object',
       'param_desc': 'String, path object (implementing os.PathLike[str]), or file-like\nobject implementing a binary readlines() function.\nAlso accepts URL. URL is not limited to S3 and GCS.\n'},
      {'param_name': 'compression',
       'param_type': 'str or dict, default ‘infer’',

## AGENDA FOR TODAY 
1. Check the validity of the data
2. Build openai functions
3. Build the graph
4. Build the Agent framework

## BUILD THE OPENAI FUNCTIONS

In [262]:
# s = "{0 or csv.QUOTE_MINIMAL, 1 or csv.QUOTE_ALL, 2 or csv.QUOTE_NONNUMERIC, 3 or csv.QUOTE_NONE}"

In [49]:
def function_text_to_req(s):
    star_idx = s.find(" *,")
    if star_idx == -1:
        return []
    req_str = s[s.find("(")+1:s.find("*")].strip()[:-1]
    req_list = [i.strip() for i in req_str.split(",")]
    return req_list

for parent in data:
    for sub_level in data[parent]['functions']:
        for func in sub_level['function_definitions']:
            func_name = "#".join(func['function_name'].split("."))
            function_calling = {"name":func_name,"descriptions":func['function_text']}
            # 
            if func['parameter_names_desc']!=[]:
                properties_dict = {}
                for params in func['parameter_names_desc']:
                    type = params['param_type']
                    if "int" in type:
                        type = "integer"
                        
                    elif (
                        "Union" in type or "list" in type
                    ) and "str" in type:
                        type = "string"
                        type_dict = {"type":type}
                    elif "str" in type:
                        type = "string"
                        type_dict = {"type":type}
                    elif "bool" in type:
                        type = "boolean"
                        type_dict = {"type":type}
                    elif "dict" in type:
                        type = "dictionary"
                        type_dict = {"type":type}
                    elif "{" in type and "}" in type and "’" in type and "‘" in type:
                        list_params = type[type.find("{")+1:type.find("}")]
                        list_params = list_params.replace("’","").replace("‘","").split(",")
                        type_dict = {"type":"string","enum":list_params}
                    else:
                        type_dict = {"type":type}
                    type_dict.update({'description':params['param_desc']})
                
                    properties_dict.update({params['param_name']:type_dict})
                
                function_calling.update({"parameters":{"type":"object","properties":properties_dict,"required":function_text_to_req(func['full_function'])}})
            func.update({"openai_function":function_calling})

In [50]:
import json
filename = "pandas_function_openai.json"

# Serialize the dictionary to a JSON string
json_data = json.dumps(data, ensure_ascii=False)

# Write the JSON string to a file with UTF-8 encoding
with open(filename, "w", encoding="utf-8") as json_file:
    json_file.write(json_data)

print("Data has been successfully written to", filename)

Data has been successfully written to pandas_function_openai.json


## BUILD GRAPH

In [217]:
data.keys()

dict_keys(['io.html', 'general_functions.html', 'series.html', 'frame.html', 'arrays.html', 'indexing.html', 'offset_frequency.html', 'window.html', 'groupby.html', 'resampling.html', 'style.html', 'plotting.html', 'options.html', 'extensions.html', 'testing.html', 'missing_value.html'])

In [51]:
import networkx as nx

pandas_graph = nx.DiGraph()

parent_names = [(data[d]['name'],{"url":data[d]['url'],"url":data[d]['url'],"type":"parent_node"}) for d in data]
pandas_graph.add_nodes_from(parent_names)

In [52]:
for parent in data:
    parent_name = data[parent]['name']
    for sub_level in data[parent]['functions']:
        for func in sub_level['function_definitions']:
            func_name = func['function_name']
            # parent_trail = parent.split(".")[0]
            pandas_graph.add_nodes_from([(func_name,{'metadata':func,'trail':f"{parent_name}","type":"function_node"})])
            pandas_graph.add_edge(parent_name,func_name)

In [53]:
# from networkx.drawing.nx_agraph import graphviz_layout
# import matplotlib.pyplot as plt
# import networkx as nx

# pos = graphviz_layout(pandas_graph, prog="dot")
# plt.figure(figsize=(20, 10))
# nx.draw(pandas_graph, pos, with_labels=True, node_size=500, node_color="skyblue", font_size=5, font_weight="bold", arrows=True)
# # plt.title(f"{router.upper()}")
# plt.show()

In [54]:
pandas_graph.nodes

NodeView(('Input/output', 'General functions', 'Series', 'DataFrame', 'pandas arrays, scalars, and data types', 'Index objects', 'Date offsets', 'Window', 'GroupBy', 'Resampling', 'Style', 'Plotting', 'Options and settings', 'Extensions', 'Testing', 'Missing values', 'pandas.read_pickle', 'pandas.DataFrame.to_pickle', 'pandas.read_table', 'pandas.DataFrame.to_csv', 'pandas.read_csv', 'pandas.read_fwf', 'pandas.read_clipboard', 'pandas.DataFrame.to_clipboard', 'pandas.read_excel', 'pandas.ExcelFile', 'pandas.io.formats.style.Styler.to_excel', 'pandas.ExcelWriter', 'pandas.DataFrame.to_excel', 'pandas.ExcelFile.parse', 'pandas.read_json', 'pandas.DataFrame.to_json', 'pandas.io.json.build_table_schema', 'pandas.json_normalize', 'pandas.read_html', 'pandas.io.formats.style.Styler.to_html', 'pandas.DataFrame.to_html', 'pandas.read_xml', 'pandas.DataFrame.to_xml', 'pandas.DataFrame.to_latex', 'pandas.io.formats.style.Styler.to_latex', 'pandas.read_hdf', 'pandas.HDFStore.append', 'pandas.HDFS

In [55]:
parent_nodes = [node for node, attributes in pandas_graph.nodes(data=True) if attributes['type']=='parent_node']

In [56]:
# for node, attributes in pandas_graph.nodes(data=True):
#     if attributes['type']=='function_node':
#         print(node,attributes)

In [57]:
parent_text_dict = {k:[] for k in parent_nodes}
for node, attributes in pandas_graph.nodes(data=True):
    if attributes['type']=='function_node':
        # parent_text_dict[attributes['trail']] += attributes['metadata']['function_text'] + "\n"
        parent_text_dict[attributes['trail']].append(attributes['metadata']['function_text'])

In [60]:
parent_text_dict['Plotting']

['Generate a matplotlib plot for visualizing clusters of multivariate data.',
 'Autocorrelation plot for time series.',
 'Bootstrap plot on mean, median and mid-range statistics.',
 'Make a box plot from DataFrame columns.',
 'Remove pandas formatters and converters.',
 'Lag plot for time series.',
 'Parallel coordinates plotting.',
 'Stores pandas plotting options.',
 'Plot a multidimensional dataset in 2D.',
 'Register pandas formatters and converters with matplotlib.',
 'Draw a matrix of scatter plots.',
 'Helper function to convert DataFrame and Series to matplotlib.table.']

## SUMMARIZATION

In [63]:
import dspy
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [64]:
class SummarizationGeneration(dspy.Signature):
    """You are given a list of descriptions of different functions separated by newline. 
    Your task is to summarize all the text into coherent summary that covers all the functions descriptions.
    Make sure that no function description is left out of the final summary. Provide a detailed summary and cover all the function descriptions"""
    
    function_descriptions = dspy.InputField(prefix="List of function descriptions: ",desc='list of function descriptions to be summarized')
    summary = dspy.OutputField(prefix="Summary: ",desc='summary of all the function descriptions')

In [65]:
summarization_llm = dspy.OpenAI(model="gpt-3.5-turbo",max_tokens=4096)
dspy.settings.configure(lm=summarization_llm)

In [66]:
filename = "parent_summary_dict.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        parent_summary_dict = json.load(json_file)

In [68]:
parent_summary_dict

{'Input/output': 'The pandas library in Python provides a wide range of functions for reading and writing data in various formats. These functions include loading pickled objects, reading delimited files, CSV files, fixed-width formatted lines, Excel files, JSON strings, HTML tables, XML documents, and more. There are also functions for working with HDFStore, feather-format objects, parquet objects, ORC objects, SAS files, SPSS files, SQL databases, Google BigQuery, and Stata files. Additionally, there are functions for creating table schemas, normalizing JSON data, rendering DataFrames to XML documents, and exporting DataFrames to Stata dta format. These functions allow for seamless data manipulation and analysis across different data sources and formats. ',
 'General functions': 'The functions provided cover a wide range of data manipulation tasks in pandas. These include unpivoting a DataFrame from wide to long format, creating pivot tables, binning values, merging DataFrames, conve

In [69]:
from tqdm import tqdm
import time
class SummarizationPipeline(dspy.Module):
    def __init__(self,parent_node,parent_text,MAX_WORDS:int=500):
        self.parent_node = parent_node
        self.parent_text = parent_text
        self.summarization = dspy.Predict(SummarizationGeneration)
        self.MAX_WORDS = MAX_WORDS
    def __call__(self,*args,**kwargs):
        return self.forward(*args,**kwargs)

    def split_description(self):
        split_s = []
        running_num_words = 0
        curr_func_string = ""
        for txt in self.parent_text:
            num_words = len(txt.split(" "))
            running_num_words+=num_words
            if running_num_words > self.MAX_WORDS:
                running_num_words = num_words
                split_s.append(curr_func_string)
                curr_func_string = txt
            else:
                curr_func_string+=txt+"\n"
        if split_s == []:
            split_s.append(curr_func_string)
        return split_s
    def forward(self):
        if len(self.parent_text) == 0:
            return ""
        split_s = self.split_description()

        summaries = ""
        pbar = tqdm(total=len(split_s),desc=f"For {self.parent_node}")
        for idx,desc in enumerate(split_s):
            summaries+=self.summarization(function_descriptions=desc).summary
            summaries+=" "
            if idx%3 == 0 and idx>0:
                print("Sleeping for 60 seconds")
                time.sleep(60)
            pbar.update(1)
        return summaries

# parent_node = "General functions"
# summ = SummarizationPipeline(parent_node,parent_text_dict[parent_node])
# summary = summ()

In [70]:
# parent_summary_dict = {k:"" for k in parent_text_dict}

for parent in parent_text_dict:
    if parent_summary_dict[parent]=="":
        print(f"Summarizing for {parent}")
        summ = SummarizationPipeline(parent,parent_text_dict[parent])
        summary = summ()
        parent_summary_dict[parent] = summary

Summarizing for Plotting


For Plotting: 100%|██████████| 1/1 [00:02<00:00,  2.80s/it]


Summarizing for Extensions


For Extensions: 100%|██████████| 1/1 [00:01<00:00,  1.98s/it]


Summarizing for Missing values


For Missing values: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it]


In [72]:
import json

# Serialize data into file:
json.dump(parent_summary_dict, open( "parent_summary_dict.json", 'w' ) )